# Creating Data Subsets
Here we will create subsets of data that can be passed into a multivariate time series regression

In [111]:
import pandas as pd
from tqdm import tqdm
import json
import numpy as np
from copy import copy
import seaborn as sns
import matplotlib.pyplot as plt

In [46]:
sales = pd.read_csv("../data/raw/sales.csv", header=0, delimiter=",")
sku = pd.read_csv("../data/raw/sku.csv", header=0, delimiter=",")
geoParams = pd.read_csv("../data/raw/geo_params.csv", header=0, delimiter=",")

Merge the geo_params and sales data files. A different subset will be generated for each item (SKU) for every geographic cluster location. The data will be stored in a dictionary with:
- key: `geoCluster_ID` and `SKU`
- value: list of `date`, `price`, and `sales`

The dictionary will be written to a json file for convenience.

### Pre-process sku file

In [55]:
sku.keys()

Index(['SKU', 'Category', 'Type', 'brandId', 'lagerUnitQuantity', 'Units',
       'trademark', 'countryOfOrigin', 'Group'],
      dtype='object')

In [79]:
sku.loc[sku['SKU'].isin( [612845, 782787, 802382, 819149, 819150] ), "Category"] = "Yoghurts"

In [81]:
# Join datasets
sales.sort_values("date", inplace=True)
sales = sales.join(geoParams.set_index("geoCluster"),on="geoCluster", rsuffix="gC")
sales = pd.merge(sales, sku, how='left', on='SKU')

#### Clean Dataset

In [82]:
sales["qty"] = sales.sales
sales = sales.drop(labels='sales', axis='columns')

In [83]:
sales = sales[["date", "SKU", "Category", "geoCluster", "Group", "price", "qty" ]]

### Dataset v1 - Non timeseries 

In [97]:
def percentage_of_criterion_in_ds(list_of_indexes, df):
    """ Return the percentage the list_of_indexes has compared to the entire df's length
    """
    percentage = 100 * sum(list_of_indexes) / len(df)
    return percentage

In [ ]:
## Exploratory Data Analysis

# # Shows you rows where qty is 0.001
# df_nts[df_nts["qty"] == 0.001]

# df_nts.describe(include='all')
# df_nts = df_nts.drop( df_nts[df_nts["price"] >= 5000].index, axis=0 )
# df_nts[df_nts["qty"] == 0.001]
# df_nts[df_nts["price"] < 50]

# df = df_nts[df_nts["Group"] == "Cheese"] 

# # Box plots price column on df
# fig, axes = plt.subplots(1,2, gridspec_kw={'width_ratios': [1, 5]}, figsize=(17,6))
# df.boxplot(column='price',ax=axes[0]);
# df.hist(column='price', ax=axes[1]);

In [129]:
df_nts = copy( sales )
df_nts = df_nts.dropna()

# Later ToDo where price is 0 fix it.
print(f"Percentage of price == 0 is {percentage_of_criterion_in_ds(df_nts['price'] == 0, df_nts)}")
# drop it
df_nts = df_nts.drop(df_nts[df_nts['price'] == 0].index, axis=0)

df_nts.to_csv("../data/processed/nts_v1.csv", index=False)

Percentage of price == 0 is 0.08415064378454352


In [92]:
# ---------- Testing ----------
df_temp = df_nts[df_nts["price"] == 0]
df_temp.to_csv("temp.csv", index=False)
# df_nts.describe(include='all')

### Dataset  - Time series

In [ ]:
# ToDo - requirements will be given by sergio later.
sales

## Depricated Code

In [4]:
SKUset = set(sales["SKU"])
salesSubsets = {}
for geoCluster in tqdm(enumerate(geoParams["geoCluster"]), total=len(geoParams["geoCluster"])):
    for sku in SKUset:
        salesSKU = sales[sales["SKU"]==sku]
        salesSKUgeoCluster = salesSKU[salesSKU["geoCluster"]==geoCluster[1]]
        key = f'{geoCluster[1]}_{sku}'
        salesSubsets[key] = salesSKUgeoCluster[["date", "price", "sales"]].values.tolist()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 515/515 [06:48<00:00,  1.26it/s]


In [5]:
salesSubsets_csv = "../data/processed/sales_subsets.csv"
try:
    with open(salesSubsets_csv, 'w') as file:
        json.dump(salesSubsets, file)

except IOError:
    print("I/O error")

I/O error


# Reload processed subsets

In [ ]:
with open(salesSubsets_csv) as file:
    salesSubsets = json.load(file)